In [ ]:
# ============================================================
#  Neural Network Surrogate for Black-Box Optimisation (BBO)
# ============================================================
#  Overview:
#  This code trains a PyTorch neural network surrogate model to
#  approximate a black-box function and identify the next best
#  query point via Monte Carlo sampling.
#
#  Core idea:
#  - The surrogate learns the relationship between inputs (X)
#    and outputs (y).
#  - Monte Carlo sampling generates many random candidates.
#  - The surrogate predicts their outputs.
#  - The best candidate (highest predicted y) is chosen as the
#    next query point.
#
#  Monte Carlo approach:
#  ---------------------
#  Since black-box functions are unknown and costly to query,
#  Monte Carlo sampling provides an efficient, probabilistic
#  method to explore the input space and estimate promising
#  regions for improvement.
#
#  Exploration vs. Exploitation Controls:
#  --------------------------------------
#  - Learning Rate (lr):
#       Higher = faster, noisier updates (more exploration)
#       Lower  = slower, stable convergence (more exploitation)
#  - Batch Size (batch_size):
#       Smaller = more stochastic gradient (exploration)
#       Larger  = smoother gradient, focus on known good regions (exploitation)
#  - Monte Carlo Samples (n_candidates):
#       More = broader coverage of search space (exploration)
#       Fewer = faster computation, focus on known regions (exploitation)
#
# ============================================================
#
# v0.02 - Full rewrite using Pytorch with clear functions for easier understanding.


In [1]:
# Mount drive for loading initial input and output files
def mount_drive():
  from google.colab import drive
  drive.mount('/content/drive')

mount_drive()

# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Mounted at /content/drive


In [2]:
# Load the initial files
def load_initial_files(function='1'):
  path = 'drive/MyDrive/data/'
  given_X = path + 'fn' + function + '_initial_inputs.npy'
  given_y = path + 'fn' + function + '_initial_outputs.npy'
  X = np.load(given_X)        # shape (n0, d)
  y = np.load(given_y)       # shape (n0,)

  return X, y

# Append new data from recent queries
def append_new_data(X, y, function):
  if function == '1':
    X = np.append(X,[[0.701438, 0.897524]], axis=0)  # Append week1 inputs
    y = np.append(y, 4.907008577514035e-55)         # Append week1 outputs
    X = np.append(X,[[0.608788, 0.208839]], axis=0)  # Append week2 inputs
    y = np.append(y, -3.407789887257921e-57)         # Append week2 outputs
    X = np.append(X,[[0.937387, 0.537998]], axis=0)  # Append week3 inputs
    y = np.append(y, -1.8248983945966781e-72)         # Append week3 outputs
    X = np.append(X,[[0.401144, 0.347316]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.000052943333535254514)         # Append week4 outputs
    X = np.append(X,[[0.815411, 0.802906]], axis=0)  # Append week5 inputs
    y = np.append(y, 2.3120136577868056e-46)         # Append week5 outputs
  elif function == '2':
    X = np.append(X,[[0.695813, 0.000000]], axis=0)  # Append week inputs
    y = np.append(y, 0.6257440183692108)         # Append week1 outputs
    X = np.append(X,[[0.969061, 0.774664]], axis=0)  # Append week2 inputs
    y = np.append(y, 0.033917749094575816)         # Append week2 outputs
    X = np.append(X,[[0.690739, 0.962975]], axis=0)  # Append week3 inputs
    y = np.append(y, 0.6440194651539941)         # Append week3 outputs
    X = np.append(X,[[0.000000, 0.982469]], axis=0)  # Append week4 inputs
    y = np.append(y, 0.011920662637457252)         # Append week4 outputs
    X = np.append(X,[[0.819992, 0.888300]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.05572035846713359)         # Append week5 outputs
  elif function == '3':
    X = np.append(X,[[0.000000, 0.000000, 0.000000]], axis=0)  # Append week1 inputs
    y = np.append(y, -0.1755204613669372)         # Append week1 outputs
    X = np.append(X,[[0.999999, 0.183024, 0.788109]], axis=0)  # Append week2 inputs
    y = np.append(y, -0.10594389054163186)         # Append week2 outputs
    X = np.append(X,[[0.937840, 0.999999, 0.405928]], axis=0)  # Append week3 inputs
    y = np.append(y, -0.05889151139553504)         # Append week3 outputs
    X = np.append(X,[[0.523777, 0.465195, 0.442681]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.005996811424753983)         # Append week4 outputs
    X = np.append(X,[[0.503614, 0.496940, 0.415922]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.0062783618309617175)         # Append week5 outputs
  elif function == '4':
    X = np.append(X,[[0.440415, 0.425453, 0.378353, 0.397106]], axis=0)  # Append week1 inputs
    y = np.append(y, 0.2601025838576061)         # Append week1 outputs
    X = np.append(X,[[0.410011, 0.415255, 0.344454, 0.438808]], axis=0)  # Append week2 inputs
    y = np.append(y, 0.3983591561199096)         # Append week2 outputs
    X = np.append(X,[[0.384038, 0.412994, 0.399499, 0.436890]], axis=0)  # Append week3 inputs
    y = np.append(y, 0.2987152140170406)         # Append week3 outputs
    X = np.append(X,[[0.391529, 0.451226, 0.356835, 0.420242]], axis=0)  # Append week4 inputs
    y = np.append(y, 0.06384840143364956)         # Append week4 outputs
    X = np.append(X,[[0.529644, 0.477474, 0.456444, 0.497979]], axis=0)  # Append week5 inputs
    y = np.append(y, -3.620742573150156)         # Append week5 outputs
  elif function == '5':
    X = np.append(X,[[0.000000, 0.827185, 0.999999, 0.999999]], axis=0)  # Append week1 inputs
    y = np.append(y, 2781.638812419282)         # Append week1 outputs
    X = np.append(X,[[0.058940, 0.195873, 0.872581, 0.998561]], axis=0)  # Append week2 inputs
    y = np.append(y, 848.8896402098709)         # Append week2 outputs
    X = np.append(X,[[0.166206, 0.999999, 0.999999, 0.999999]], axis=0)  # Append week3 inputs
    y = np.append(y, 4444.256290210307)         # Append week3 outputs
    X = np.append(X,[[0.259704, 0.866240, 0.999999, 0.078304]], axis=0)  # Append week4 inputs
    y = np.append(y, 837.6996715082641)         # Append week4 outputs
    X = np.append(X,[[0.405221, 0.818629, 0.840579, 0.874304]], axis=0)  # Append week5 inputs
    y = np.append(y, 848.6842411794586)         # Append week5 outputs
  elif function == '6':
    X = np.append(X,[[0.464910, 0.242338, 0.574752, 0.999999, 0.000000]], axis=0)  # Append week1 inputs
    y = np.append(y, -0.5265043497038704)         # Append week1 outputs
    X = np.append(X,[[0.537065, 0.317653, 0.626041, 0.898937, 0.148218]], axis=0)  # Append week2 inputs
    y = np.append(y, -0.41510731628352715)         # Append week2 outputs
    X = np.append(X,[[0.455585, 0.174069, 0.999999, 0.999999, 0.292903]], axis=0)  # Append week3 inputs
    y = np.append(y, -1.0104998624615082)         # Append week3 outputs
    X = np.append(X,[[0.525055, 0.369255, 0.490228, 0.765387, 0.025091]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.3481623700173726)         # Append week4 outputs
    X = np.append(X,[[0.541275, 0.517331, 0.640338, 0.739537, 0.379138]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.6094732138365111)         # Append week5 outputs
  elif function == '7':
    X = np.append(X,[[0.000000, 0.247036, 0.408965, 0.217149, 0.377534, 0.746590]], axis=0)  # Append week1 inputs
    y = np.append(y, 2.3034568430941222)         # Append week1 outputs
    X = np.append(X,[[0.000000, 0.181841, 0.435826, 0.062982, 0.361647, 0.858489]], axis=0)  # Append week2 inputs
    y = np.append(y, 1.3279380639726712)         # Append week2 outputs
    X = np.append(X,[[0.000000, 0.166738, 0.211207, 0.080984, 0.381370, 0.746608]], axis=0)  # Append week3 inputs
    y = np.append(y, 1.433484749556216)         # Append week3 outputs
    X = np.append(X,[[0.000000, 0.198493, 0.750365, 0.247059, 0.370826, 0.980737]], axis=0)  # Append week4 inputs
    y = np.append(y, 1.239424289285777)         # Append week4 outputs
    X = np.append(X,[[0.456843, 0.374177, 0.452545, 0.525118, 0.456228, 0.722847]], axis=0)  # Append week5 inputs
    y = np.append(y, 0.8787401362260749)         # Append week5 outputs
  elif function == '8':
    X = np.append(X,[[0.060275, 0.000000, 0.134973, 0.000000, 0.999999, 0.404343, 0.057755, 0.516689]], axis=0)  # Append week1 inputs
    y = np.append(y, 9.8814582425914)         # Append week1 outputs
    X = np.append(X,[[0.122654, 0.153991, 0.162413, 0.045600, 0.999999, 0.536650, 0.260832, 0.932950]], axis=0)  # Append week2 inputs
    y = np.append(y, 9.9450768395815)         # Append week2 outputs
    X = np.append(X,[[0.085442, 0.318585, 0.000000, 0.239064, 0.999999, 0.927570, 0.142651, 0.932950]], axis=0)  # Append week3 inputs
    y = np.append(y, 9.6920435401985)         # Append week3 outputs
    X = np.append(X,[[0.266427, 0.000000, 0.185600, 0.000000, 0.999999, 0.210401, 0.179882, 0.156373]], axis=0)  # Append week4 inputs
    y = np.append(y, 9.7659726871796)         # Append week4 outputs
    X = np.append(X,[[0.514029, 0.459555, 0.534367, 0.406509, 0.744510, 0.663310, 0.578111, 0.756422]], axis=0)  # Append week5 inputs
    y = np.append(y, 8.6884084301446)         # Append week5 outputs


  return X, y

# Normalise X and y
def normalise_data(X, y):
  x_scaler = StandardScaler()
  Xn = x_scaler.fit_transform(X)
  y_mean = y.mean()
  y_std = y.std() if y.std() > 0 else 1.0
  yn = (y - y_mean) / y_std   # Normalize target manually (avoid sklearn overhead)

  return Xn, yn, y_mean, y_std, x_scaler

In [5]:
# Define Neural Network surrogate model
class SurrogateNN(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.2):
        super(SurrogateNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            #nn.Dropout(dropout_rate),
            nn.Linear(64, 32),
            nn.ReLU(),
            #nn.Dropout(dropout_rate),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# Train Surrogate model with optional mini batching
def train_surrogate(X_train, y_train, input_dim, lr=0.01, epochs=300, batch_size=None):
    """
    Trains the neural network surrogate.
    If batch_size is None, uses full-batch training.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SurrogateNN(input_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    X_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

    if batch_size is not None:
        dataset = TensorDataset(X_t, y_t)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        loader = [(X_t, y_t)]  # full-batch fallback

    for epoch in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()

    return model

# Monte Carlo Sampling to Find Next Query Point
def suggest_next_query(model, input_dim, x_scaler, y_mean, y_std, n_candidates=10000):
    """
    Uses Monte Carlo sampling to suggest the next query point.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # Generate random candidate points uniformly in [0,1]^d
    candidates = np.random.rand(n_candidates, input_dim)

    #Normalise candidates using fitted scaler
    candidates_norm = x_scaler.transform(candidates)

    # Predict for normalised candidates
    with torch.no_grad():
        preds_norm = model(torch.tensor(candidates_norm, dtype=torch.float32).to(device)).cpu().numpy()

    # Select candidate with highest predicted value
    best_idx = np.argmax(preds_norm)
    x_next_norm = candidates_norm[best_idx]
    y_pred_norm = preds_norm[best_idx]

    # Denormalise back to original scale
    x_next = x_scaler.inverse_transform(x_next_norm.reshape(1, -1)).flatten()
    y_pred = y_pred_norm * y_std + y_mean

    return x_next, y_pred

In [68]:
# Decide function and batch_size
func = '8'       # number between 1 and 9 representing BBO function
batch_size = 8   # number or None

# Load data and normalise
X, y = load_initial_files(function=func)
X, y = append_new_data(X, y, function=func)
Xn, yn, y_mean, y_std, x_scaler = normalise_data(X, y)

print(max(y))
print(max(yn))

# Calcualte input shape
input_dim = Xn.shape[1]

model = train_surrogate(Xn, yn, input_dim, lr=0.01, epochs=500, batch_size=batch_size)

x_next, y_pred = suggest_next_query(model, input_dim, x_scaler, y_mean, y_std, n_candidates=50000)

print("\nFunction:", func)
print("\nSuggested next input (x_next):", '-'.join(map(str, np.round(x_next, 6))))
print("\nPredicted output estimate:", float(y_pred[0]))



9.9450768395815
1.8148710531287375

Function: 8

Suggested next input (x_next): 0.050667-0.157535-0.098203-0.507794-0.927913-0.283519-0.019433-0.091501

Predicted output estimate: 10.014274882683205


Function: 1

Suggested next input (x_next): 0.147927-0.999351

Predicted output estimate: 0.004738142161169105

Function: 2

Suggested next input (x_next): 0.576225-0.995582

Predicted output estimate: 1.0778257373391364

Function: 3

Suggested next input (x_next): 0.306294-0.000901-0.737840

Predicted output estimate: 0.03597608387388612

Function: 4

Suggested next input (x_next): 0.362616-0.390935-0.358916-0.408803

Predicted output estimate: 0.4543251119036995

Function: 5

Suggested next input (x_next): 0.379668-0.980190-0.997527-0.939278

Predicted output estimate: 4366.357278991082

Function: 6

Suggested next input (x_next): 0.377302-0.427069-0.554688-0.835249-0.037887

Predicted output estimate: -0.23066364279575358

Function: 7

Suggested next input (x_next): 0.002100-0.111359-0.320991-0.376242-0.229457-0.955090

Predicted output estimate: 2.464740596873976

Function: 8

Suggested next input (x_next): 0.026980-0.613170-0.025877-0.359845-0.896836-0.008212-0.028519-0.131032

Predicted output estimate: 10.435474844266569

